# test embeddings

plot tsne of the embeddings extracted by the CNN model trained on the dataset.

-> write a module for input generation. given the folder and file names get NxTxM matrix
-> each data point, load model in eval mode. 
-> forward the given 

In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision.transforms as transforms
import sys
from ret_benchmark.config import cfg
from ret_benchmark.modeling import build_model
import os
import numpy as np
import re
from sklearn.manifold import TSNE
import pickle
import matplotlib.pyplot as plt

os.environ['CUDA_VISIBLE_DEVICES'] = '3' # gpu to run the code
sys.path.insert(1, '/data/home/shruti/voxceleb/motion_signature/research-ms-loss-master/')
config_path = 'configs/ms_resnet50_fabnet_vox_test.yaml'
cache_emb_file = 'ms_resnet50_fabnet_vox_test' # this file is cache for fabnet metric learning embeddings on leader data


In [2]:
# get all the embedding
def load_obj(fldr, name ):
    if not os.path.exists(os.path.join(fldr, name + '.pkl')):
        return None
    with open(os.path.join(fldr, name + '.pkl'), 'rb') as f:
        return pickle.load(f)

# save the obj
def save_obj(obj, fldr, name ):
    os.makedirs(fldr, exist_ok=True)
    with open(os.path.join(fldr, name + '.pkl'), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

## load model

In [3]:
def load_model(cfgpath):
    
    cfg.merge_from_file(cfgpath)
    
    # load the model
    model = build_model(cfg)
    device = torch.device(cfg.MODEL.DEVICE)
    model.to(device)
    model.eval()
    return model

In [4]:
model = load_model(config_path)

Resuming from model ...


## load data and labels

In [5]:
subfldrs = np.array(['bo_faceswap', 'bo_UWfake', 'diff_test', 'dt_faceswap', 'hc_faceswap', 'bs_faceswap',
                        'ew_faceswap', 'dt_imposter', 'hc_imposter', 'bs_imposter', 'ew_imposter', 'bo_imposter', 'trump_fake'])

def get_fabnet_ldrs_data():
        
    # read mean and std files
    means = np.reshape(np.load(cfg.INPUT.MEAN), (1, -1))
    stds = np.reshape(np.load(cfg.INPUT.STD), (1, -1))

    # the test files of leaders
    root = os.path.dirname(cfg.DATA.TEST_IMG_SOURCE)
    path_list = []
    label_list = []
    print(cfg.DATA.TRAIN_IMG_SOURCE)
    with open(cfg.DATA.TRAIN_IMG_SOURCE, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
            
    print(cfg.DATA.TEST_IMG_SOURCE)
    with open(cfg.DATA.TEST_IMG_SOURCE, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
    
    test_file = cfg.DATA.TEST_IMG_SOURCE.replace('_val', '_test')
    print(test_file)
    with open(test_file, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
    
    # add the files from 
    ofset = np.max(label_list) + 1
    for j in range(len(subfldrs)):
        _path = [os.path.join(subfldrs[j], f) for f in os.listdir(os.path.join(root, subfldrs[j])) if f.endswith('.npy')]
        _label = list(np.zeros((len(_path), ), dtype='int')+ofset+j)
        path_list.extend(_path)
        label_list.extend(_label)
    
    data = {}
    data['mean'] = means
    data['std'] = stds
    data['root'] = root
    data['path_list'] = path_list
    data['label_list'] = label_list
    
    return data

def get_vgg_ldrs_data():
        
    # read mean and std files
    means = np.reshape(np.load(cfg.INPUT.MEAN), (1, -1))
    stds = np.reshape(np.load(cfg.INPUT.STD), (1, -1))

    # the test files of leaders
    root = os.path.dirname(cfg.DATA.TEST_IMG_SOURCE)
    path_list = []
    label_list = []
    print(cfg.DATA.TRAIN_IMG_SOURCE)
    with open(cfg.DATA.TRAIN_IMG_SOURCE, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
            
    print(cfg.DATA.TEST_IMG_SOURCE)
    with open(cfg.DATA.TEST_IMG_SOURCE, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
    
    test_file = cfg.DATA.TEST_IMG_SOURCE.replace('_val', '_test')
    print(test_file)
    with open(test_file, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
    
    # add the files from 
    ofset = np.max(label_list) + 1
    for j in range(len(subfldrs)):
        _path = [f for f in os.listdir(root) if f.endswith('.npy') and f[:len(subfldrs[j])+1] == subfldrs[j]+'_']
        _label = list(np.zeros((len(_path), ), dtype='int')+ofset+j)
        path_list.extend(_path)
        label_list.extend(_label)
    
    data = {}
    data['mean'] = means
    data['std'] = stds
    data['root'] = root
    data['path_list'] = path_list
    data['label_list'] = label_list
    
    return data

In [7]:
data_files = get_fabnet_ldrs_data()
#data_files = get_vgg_ldrs_data()
print(len(data_files['path_list']))

/data/home/shruti/voxceleb/fabnet/leaders/leaders_150_train.txt
/data/home/shruti/voxceleb/fabnet/leaders/leaders_150_val.txt
/data/home/shruti/voxceleb/fabnet/leaders/leaders_150_test.txt
4329


# get metric embedding

for every video file:
    1) load the npy
    2) for every 150 window with 5 frame step, pass it through the model and get the embedding
    3) in the save the embedding and the label in a dictionary
    4) save the embeddings 

In [8]:
np.unique(data_files['label_list'])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22])

In [9]:
def feature_extract(infile):
    
    st = 5; emb_sz = cfg.MODEL.HEAD.DIM
    try:
        fab_emb = np.load(infile)
    except Exception as e:
        print(e)
        return None
    
    # if not enough time window
    if len(fab_emb)<cfg.INPUT.FRAME_LENGTH:
        return None
    
    if len(fab_emb.shape)>2:
        fab_emb = fab_emb[:, :, 0].copy()
    
    fab_emb = fab_emb/np.linalg.norm(fab_emb, axis=1, keepdims=True)
    fab_emb = (fab_emb-data_files['mean'])/data_files['std']
    
    feat_rng = np.arange(0, len(fab_emb)-cfg.INPUT.FRAME_LENGTH+1, st)
    out_feat = np.zeros((len(feat_rng), emb_sz))
    print(f' {out_feat.shape}')
    for i in range(len(feat_rng)):
        cur_data = fab_emb[feat_rng[i]:feat_rng[i]+cfg.INPUT.FRAME_LENGTH, :].copy()
        
        #transform
        cur_data = np.reshape(cur_data, (1, 1, cur_data.shape[0], cur_data.shape[1]))
        tensor_data = torch.from_numpy(cur_data).cuda().float()

        # pass the model
        with torch.no_grad():
            out_feat[i, :] = model(tensor_data).data.cpu().numpy().ravel()
    
    return out_feat

In [11]:
int2str_lbl = {0:'bo', 1:'br', 2:'bs', 3:'cb', 4:'dt_week', 4:'dt_rndm', 5:'ew', 6:'hc', 7:'jb', 8:'kh', 9:'pb', 
               10:'bo_faceswap', 11:'bo_UWfake', 12:'diff_test', 13:'dt_faceswap', 14:'hc_faceswap', 15:'bs_faceswap',
               16:'ew_faceswap', 17:'dt_imposter', 18:'hc_imposter', 19:'bs_imposter', 20:'ew_imposter', 
               21:'bo_imposter', 22:'trump_fake'}

for lbl in int2str_lbl.keys():
    
    # for every embedding file
    cur_dict = load_obj( f'obj/{cache_emb_file}', f'{cfg.INPUT.FRAME_LENGTH}_{int2str_lbl[lbl]}')    
    if cur_dict is None:

        cur_dict = {}
        cur_files = np.array(data_files['path_list'])[np.array(data_files['label_list'])==lbl]
        for i in range(len(cur_files)): #

            print(cur_files[i], end=' ')
            # 
            cur_feat = feature_extract(os.path.join(data_files['root'], cur_files[i]))
            if cur_feat is not None:
                cur_dict[cur_files[i]] = cur_feat

        save_obj(cur_dict,  f'obj/{cache_emb_file}', f'{cfg.INPUT.FRAME_LENGTH}_{int2str_lbl[lbl]}')

bo/--QxAtXZJpQ_0.npy  (262, 512)
bo/--QxAtXZJpQ_1.npy  (296, 512)
bo/--QxAtXZJpQ_2.npy  (295, 512)
bo/--QxAtXZJpQ_3.npy  (295, 512)


KeyboardInterrupt: 